# Python API Example - Historical FFA Price Release Data Import and Storage in Dataframe
## Importing FFA Price Data into a Pandas DataFrame

This guide is designed to provide an example of how to access the Spark API:
- The path to your client credentials is the only input needed to run this script (just before Section 2)
- This script has been designed to display the raw outputs of requests from the API, and then shows you how to format those outputs to enable easy reading and analysis
- This script can be copied and pasted by customers for quick use of the API
- Once comfortable with the process, you can change the variables that are called to produce your own custom analysis products. (Section 2 onwards in this guide).

__N.B. This guide is just for FFA Price release data. If you're looking for other API data products (such as Freight routes or Netbacks), please refer to their according code example files.__ 



### Have any questions?

If you have any questions regarding our API, or need help accessing specific datasets, please contact us at:

__data@sparkcommodities.com__

or refer to our API website for more information about this endpoint:
https://www.sparkcommodities.com/api/request/contracts.html

## 1. Importing Data

Here we define the functions that allow us to retrieve the valid credentials to access the Spark API.

This section can remain unchanged for most Spark API users.

In [1]:
# importing packages for calling the API
import json
import os
import sys
from base64 import b64encode
from pprint import pprint
from urllib.parse import urljoin
from datetime import datetime
import pandas as pd

try:
    from urllib import request, parse
    from urllib.error import HTTPError
except ImportError:
    raise RuntimeError("Python 3 required")

In [2]:
# Defining functions for API request


API_BASE_URL = "https://api.sparkcommodities.com"


def retrieve_credentials(file_path=None):
    """
    Find credentials either by reading the client_credentials file or reading
    environment variables
    """
    if file_path is None:
        client_id = os.getenv("SPARK_CLIENT_ID")
        client_secret = os.getenv("SPARK_CLIENT_SECRET")
        if not client_id or not client_secret:
            raise RuntimeError(
                "SPARK_CLIENT_ID and SPARK_CLIENT_SECRET environment vars required"
            )
    else:
        # Parse the file
        if not os.path.isfile(file_path):
            raise RuntimeError("The file {} doesn't exist".format(file_path))

        with open(file_path) as fp:
            lines = [l.replace("\n", "") for l in fp.readlines()]

        if lines[0] in ("clientId,clientSecret", "client_id,client_secret"):
            client_id, client_secret = lines[1].split(",")
        else:
            print("First line read: '{}'".format(lines[0]))
            raise RuntimeError(
                "The specified file {} doesn't look like to be a Spark API client "
                "credentials file".format(file_path)
            )

    print(">>>> Found credentials!")
    print(
        ">>>> Client_id={}, client_secret={}****".format(client_id, client_secret[:5])
    )

    return client_id, client_secret


def do_api_post_query(uri, body, headers):
    """
    OAuth2 authentication requires a POST request with client credentials before accessing the API. 
    This POST request will return an Access Token which will be used for the API GET request.
    """
    url = urljoin(API_BASE_URL, uri)

    data = json.dumps(body).encode("utf-8")

    # HTTP POST request
    req = request.Request(url, data=data, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 201, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def do_api_get_query(uri, access_token):
    """
    After receiving an Access Token, we can request information from the API.
    """
    url = urljoin(API_BASE_URL, uri)

    headers = {
        "Authorization": "Bearer {}".format(access_token),
        "Accept": "application/json",
    }

    # HTTP POST request
    req = request.Request(url, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 200, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def get_access_token(client_id, client_secret):
    """
    Get a new access_token. Access tokens are the thing that applications use to make
    API requests. Access tokens must be kept confidential in storage.

    # Procedure:

    Do a POST query with `grantType` and `scopes` in the body. A basic authorization
    HTTP header is required. The "Basic" HTTP authentication scheme is defined in
    RFC 7617, which transmits credentials as `clientId:clientSecret` pairs, encoded
    using base64.
    """

    # Note: for the sake of this example, we choose to use the Python urllib from the
    # standard lib. One should consider using https://requests.readthedocs.io/

    payload = "{}:{}".format(client_id, client_secret).encode()
    headers = {
        "Authorization": b64encode(payload).decode(),
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    body = {
        "grantType": "clientCredentials",
        "scopes": "read:lng-freight-prices,read:routes",
    }

    content = do_api_post_query(uri="/oauth/token/", body=body, headers=headers)

    print(
        ">>>> Successfully fetched an access token {}****, valid {} seconds.".format(
            content["accessToken"][:5], content["expiresIn"]
        )
    )

    return content["accessToken"]

## Defining Fetch Request

Here is where we define what type of data we want to fetch from the API.

In my fetch request, I use the URL:

__uri="/v1.0/contracts/"__

This is to query contract price data specifically. Other data products (such as shipping route costs) require different URL's in the fetch request (refer to other Python API examples).

In [3]:
# Defining function for collecting the list of contracts
def list_contracts(access_token):
    """
    Fetch available contracts. Return contract ticker symbols

    # Procedure:

    Do a GET query to /v1.0/contracts/ with a Bearer token authorization HTTP header.
    """
    content = do_api_get_query(uri="/v1.0/contracts/", access_token=access_token)

    print(">>>> All the contracts you can fetch")
    tickers = []
    for contract in content["data"]:
        print(contract["fullName"])
        tickers.append(contract["id"])

    return tickers

## N.B. Credentials

Here we call the above functions, and input the file path to our credentials.

N.B. You must have downloaded your client credentials CSV file before proceeding. Please refer to the API documentation if you have not dowloaded them already. Instructions for downloading your credentials can be found here:

https://api.sparkcommodities.com/redoc#section/Authentication/Create-an-Oauth2-Client


The code then prints the available prices that are callable from the API, and their corresponding Python ticker names are displayed as a list at the bottom of the Output.

In [4]:
# Insert file path to your client credentials here
client_id, client_secret = retrieve_credentials(file_path="/tmp/client_credentials.csv")

# Authenticate:
access_token = get_access_token(client_id, client_secret)

# Fetch all contracts:
tickers = list_contracts(access_token)


print(tickers)

>>>> Found credentials!
>>>> Client_id=875f483b-19de-421a-8e9b-dceff6703e83, client_secret=6cdf8****
>>>> Successfully fetched an access token eyJhb****, valid 604799 seconds.
>>>> All the contracts you can fetch
Spark25F Pacific 160 TFDE
Spark30F Atlantic 160 TFDE
Spark25S Pacific
Spark25Fo Pacific
Spark25FFA Pacific
Spark25FFAYearly Pacific
Spark30S Atlantic
Spark30Fo Atlantic
Spark30FFA Atlantic
Spark30FFAYearly Atlantic
SparkNWE DES 1H
SparkNWE-B 1H
SparkNWE DES 2H
SparkNWE-B 2H
SparkNWE-B F
SparkNWE DES F
SparkNWE-B Fo
SparkNWE DES Fo
SparkNWE-DES-Fin Monthly
SparkNWE-Fin Monthly
SparkSWE-B F
SparkSWE DES F
SparkSWE-B Fo
SparkSWE DES Fo
SparkSWE-DES-Fin Monthly
SparkSWE-Fin Monthly
['spark25f', 'spark30f', 'spark25s', 'spark25fo', 'spark25ffa-monthly', 'spark25ffa-yearly', 'spark30s', 'spark30fo', 'spark30ffa-monthly', 'spark30ffa-yearly', 'sparknwe-1h', 'sparknwe-b-1h', 'sparknwe-2h', 'sparknwe-b-2h', 'sparknwe-b-f', 'sparknwe-f', 'sparknwe-b-fo', 'sparknwe-fo', 'sparknwe-des-fin

## 2. Latest Price Release

Here we call the latest price release and print it in a readable format. This is done using the URL:

__/v1.0/contracts/{contract_ticker_symbol}/price-releases/latest/__

'tickers' refers to the printed list above, so we can see that:
- 'tickers[4]' refers to 'Spark25FFA' 
- 'tickers[8]' refers to 'Spark30FFA' 

We then save the entire dataset as a local variable called 'my_dict'.

__N.B. The first two tickers, 'spark25f' and 'spark30f', are deprecated. Historical data for these tickers are available up until 2022-04-01 (yyyy-mm-dd)__

For more information on API updates, please refer to the API documentation:

https://api.sparkcommodities.com/redoc#section/API-Changelog

In [5]:
## Defining the function


def fetch_latest_price_releases(access_token, ticker):
    """
    For a contract, fetch then display the latest price release

    # Procedure:

    Do GET queries to /v1.0/contracts/{contract_ticker_symbol}/price-releases/latest/
    with a Bearer token authorization HTTP header.
    """
    content = do_api_get_query(
        uri="/v1.0/contracts/{}/price-releases/latest/".format(ticker),
        access_token=access_token,
    )

    release_date = content["data"]["releaseDate"]

    print(">>>> Get latest price release for {}".format(ticker))
    print("release date =", release_date)

    data_points = content["data"]["data"][0]["dataPoints"]

    return content["data"]


## Calling that function and storing the output

# Here we store the latest Spark25FFA release called from the API

my_dict = fetch_latest_price_releases(access_token, tickers[4])

>>>> Get latest price release for spark25ffa-monthly
release date = 2025-01-14


In [6]:
# Shows how the raw output is formatted
my_dict

{'id': 20250114,
 'contractId': 'spark25ffa-monthly',
 'releaseDate': '2025-01-14',
 'previousPriceRelease': {'id': 20250113, 'releaseDate': '2025-01-13'},
 'nextPriceRelease': {'id': 20250115, 'releaseDate': '2025-01-15'},
 'assessmentWindowClosedAt': '2025-01-14T17:00:00Z',
 'assessmentWindowOpenedAt': '2025-01-14T15:30:00Z',
 'data': [{'revisionNumber': 0,
   'revisionPublishedAt': '2025-01-14T16:47:22.843810Z',
   'numberOfAssessors': None,
   'dataPoints': [{'index': 0,
     'deliveryPeriod': {'type': 'month',
      'startAt': '2025-01-01',
      'endAt': '2025-01-31',
      'name': 'M+0',
      'lastAssessmentDate': '2025-01-30'},
     'yourAssessedPrice': None,
     'derivedPrices': {'usdPerDay': {'spark': '21500',
       'sparkMin': '21500',
       'sparkMax': '21750',
       'portfolioPlayer': None,
       'portfolioPlayerMin': None,
       'portfolioPlayerMax': None,
       'shipOwner': None,
       'shipOwnerMin': None,
       'shipOwnerMax': None},
      'usdPerMMBtu': {'sp

## 3. Historical Prices

Here we perform a similar task, but with historical prices instead. This is done using the URL:

__/v1.0/contracts/{contract_ticker_symbol}/price-releases/{limit}{offset}__

First we define the function that imports the data from the Spark API.

We then call that function, and define 2 parameters:

- 'tickers': which ticker do you want to call.
    - We define the variable 'my_ticker' after the function definition, and set this to 'tickers[4]' which corresponds to Spark25FFA
    - Alter this variable to whatever price product you need.

- 'limit': this allows you to control how many datapoints you want to call. Here we use 'limit=10', which means we have called the last 10 datapoints (the Spark25FFA data for the last 1000 business days).
    - For __Premium__ Users, alter this limit to however many datapoints you need.
    - For __Trial__ Users, the limit parameter must not exceed 14 datapoints, as historical data is limited to 2 weeks for this plan.
    - If you ask for more datapoints than is available, the API will just retrieve all the data available (as seen below)


We save the output as a local variable called 'my_dict_hist'

In [7]:
def fetch_historical_price_releases(access_token, ticker, limit, offset=None):
    """
    For a selected contract, this endpoint returns all the Price Releases you can
    access according to your current subscription, ordered by release date descending.

    **Note**: Unlimited access to historical data and full forward curves is only
    available to those with Premium access. Get in touch to find out more.

    **Params**

    limit: optional integer value to set an upper limit on the number of price
           releases returned by the endpoint.

    offset: optional integer value to set from where to start returning data.
            Default is 0.

    # Procedure:

    Do GET queries to /v1.0/contracts/{contract_ticker_symbol}/price-releases/
    with a Bearer token authorization HTTP header.
    """
    print(">>>> Get price releases for {}".format(ticker))

    query_params = "?limit={}".format(limit)
    if offset is not None:
        query_params += "&offset={}".format(offset)

    content = do_api_get_query(
        uri="/v1.0/contracts/{}/price-releases/{}".format(ticker, query_params),
        access_token=access_token,
    )

    my_dict = content["data"]

    for release in content["data"]:
        release_date = release["releaseDate"]

        print("- release date =", release_date)

        data_points = release["data"][0]["dataPoints"]

    return my_dict

### N.B. Plan Limits

__Premium__ Plan users have __no__ limits on historical data.

__Trial__ Plan users only have access to the latest 2 weeks worth of historical data. Therefore the limit parameter cannot exceed 14.

In [8]:
### Define which price product you want to retrieve
my_ticker = tickers[4]


# Call the function, and set limit=1000 to call 1000 datapoints

my_dict_hist = fetch_historical_price_releases(access_token, my_ticker, limit=10)


# View data in its raw format

my_dict_hist 

>>>> Get price releases for spark25ffa-monthly
- release date = 2025-01-14
- release date = 2025-01-13
- release date = 2025-01-10
- release date = 2025-01-09
- release date = 2025-01-08
- release date = 2025-01-07
- release date = 2025-01-06
- release date = 2025-01-03
- release date = 2025-01-02
- release date = 2024-12-31


[{'id': 20250114,
  'contractId': 'spark25ffa-monthly',
  'releaseDate': '2025-01-14',
  'previousPriceRelease': {'id': 20250113, 'releaseDate': '2025-01-13'},
  'nextPriceRelease': {'id': 20250115, 'releaseDate': '2025-01-15'},
  'assessmentWindowClosedAt': '2025-01-14T17:00:00Z',
  'assessmentWindowOpenedAt': '2025-01-14T15:30:00Z',
  'data': [{'revisionNumber': 0,
    'revisionPublishedAt': '2025-01-14T16:47:22.843810Z',
    'numberOfAssessors': None,
    'dataPoints': [{'index': 0,
      'deliveryPeriod': {'type': 'month',
       'startAt': '2025-01-01',
       'endAt': '2025-01-31',
       'name': 'M+0',
       'lastAssessmentDate': '2025-01-30'},
      'yourAssessedPrice': None,
      'derivedPrices': {'usdPerDay': {'spark': '21500',
        'sparkMin': '21500',
        'sparkMax': '21750',
        'portfolioPlayer': None,
        'portfolioPlayerMin': None,
        'portfolioPlayerMax': None,
        'shipOwner': None,
        'shipOwnerMin': None,
        'shipOwnerMax': None},

In [9]:
# Check the amount of datapoints (Spark25FFA) releases
len(my_dict_hist)

10

## Function to call data and store as a DataFrame

__N.B. The structure of the called data is slightly differs based on the ticker used. For example, the formatting method below is applicable to the FFA data, but would have to be slightly altered if a different ticker were used instead (for example, please see our 'spark_api_historical_spot_prices' tutorial script to see how to format the historical spot price data instead).__

In [12]:
# Defining the function
def fetch_ffa_prices(my_tick, my_lim):
    print(my_tick)

    my_dict_hist = fetch_historical_price_releases(access_token, my_tick, limit=my_lim)

    release_dates = []

    period_start = []
    period_end = []
    period_name = []
    cal_month = []

    ticker = []

    usd_day = []

    day_min = []
    day_max = []

    for release in my_dict_hist:
        release_date = release["releaseDate"]

        print("- release date =", release_date)

        data = release["data"]

        for d in data:
            data_points = d["dataPoints"]
            for data_point in data_points:
                period_start_at = data_point["deliveryPeriod"]["startAt"]
                period_start.append(period_start_at)
                period_end_at = data_point["deliveryPeriod"]["endAt"]
                period_end.append(period_end_at)
                period_name.append(data_point["deliveryPeriod"]["name"])

                release_dates.append(release_date)
                # release_dates.append(datetime.strptime(release_date, '%Y-%m-%d'))
                ticker.append(release["contractId"])
                cal_month.append(
                    datetime.strptime(period_start_at, "%Y-%m-%d").strftime("%b-%Y")
                )

                usd_day.append(int(data_point["derivedPrices"]["usdPerDay"]["spark"]))
                day_min.append(
                    int(data_point["derivedPrices"]["usdPerDay"]["sparkMin"])
                )
                day_max.append(
                    int(data_point["derivedPrices"]["usdPerDay"]["sparkMax"])
                )

    historical_df = pd.DataFrame(
        {
            "Release Date": release_dates,
            "ticker": ticker,
            "Period Name": period_name,
            "Period Start": period_start,
            "Period End": period_end,
            "Calendar Month": cal_month,
            "Spark": usd_day,
            "SparkMin": day_min,
            "SparkMax": day_max,
        }
    )

    historical_df['Release Date'] = pd.to_datetime(historical_df['Release Date'],format='%Y-%m-%d')

    return historical_df

# Call those functions for Spark30FFA and Spark25FFA

We call the function defined above and create two dataframes:

- spark25ffa - storing all historical Spark25FFA data
- spark30ffa - storing all historical Spark30FFA data

In [13]:
spark25ffa = fetch_ffa_prices(tickers[4], 10)

spark30ffa = fetch_ffa_prices(tickers[8], 10)

spark25ffa-monthly
>>>> Get price releases for spark25ffa-monthly
- release date = 2024-10-16
- release date = 2024-10-15
- release date = 2024-10-14
- release date = 2024-10-11
- release date = 2024-10-10
- release date = 2024-10-09
- release date = 2024-10-08
- release date = 2024-10-07
- release date = 2024-10-04
- release date = 2024-10-03
- release date = 2024-10-16
- release date = 2024-10-15
- release date = 2024-10-14
- release date = 2024-10-11
- release date = 2024-10-10
- release date = 2024-10-09
- release date = 2024-10-08
- release date = 2024-10-07
- release date = 2024-10-04
- release date = 2024-10-03
spark30ffa-monthly
>>>> Get price releases for spark30ffa-monthly
- release date = 2024-10-16
- release date = 2024-10-15
- release date = 2024-10-14
- release date = 2024-10-11
- release date = 2024-10-10
- release date = 2024-10-09
- release date = 2024-10-08
- release date = 2024-10-07
- release date = 2024-10-04
- release date = 2024-10-03
- release date = 2024-10-16


# Save as separate Excel Spreadsheets 

For those more comfortable sorting through categorical data in Excel, we can easily export these DataFrames as separate Excel Files.

These Excel files will include the raw data. From here, the data can be grouped or transformed into a Pivot Table in Excel, so that data can be filtered by Release Date or other suitable variables.

In [14]:
save_to_excel = False # change value as you see fit

if save_to_excel == True:
    path_25ffa = "/tmp/Spark25ffa_historical.xlsx"
    spark25ffa.to_excel(path_25ffa)

    path_30ffa = "/tmp/Spark30ffa_historical.xlsx"
    spark30ffa.to_excel(path_30ffa)

# Analytics using Python 

Alternatively, we can group and analyse the data here in the script.

Below are some examples of how this data can be filtered, grouped and analysed.

In [15]:
# Filtering by release date
spark30_groups = spark30ffa.groupby(["Release Date"])

# Get the latest release date
releases = list(spark30_groups.groups.keys())

# Using this date, get the latest FFA prices
spark30latest_ffa = spark30_groups.get_group(releases[-1])
spark30latest_ffa.head()

/var/folders/v3/5pn2lssn077ch9xm2rttdmym0000gn/T/ipykernel_95286/1131647169.py:8: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  spark30latest_ffa = spark30_groups.get_group(releases[-1])


,Release Date,ticker,Period Name,Period Start,Period End,Calendar Month,Spark,SparkMin,SparkMax
0,2024-10-16,spark30ffa-monthly,M+0,2024-10-01,2024-10-31,Oct-2024,42750,42000,44500
1,2024-10-16,spark30ffa-monthly,M+1,2024-11-01,2024-11-30,Nov-2024,42500,40000,43000
2,2024-10-16,spark30ffa-monthly,M+2,2024-12-01,2024-12-31,Dec-2024,47750,42000,52000
3,2024-10-16,spark30ffa-monthly,M+3,2025-01-01,2025-01-31,Jan-2025,51000,43000,55000
4,2024-10-16,spark30ffa-monthly,M+4,2025-02-01,2025-02-28,Feb-2025,46000,43000,48000


# Analytics Gallery
Want to gain market insights using our data?

Take a look at our [Analytics Gallery](https://www.sparkcommodities.com/api/code-examples/analytics-examples.html) on the Spark API website, which includes:

- __Spark30 FFAs Seasonality Chart__ - Compare how Spark30FFA prices for a given contract month evolve over time, and see how similar contract months in previous years compare (e.g. Dec22 vs Dec23 vs Dec24).

View our FFAs Seasonality Chart [here](https://www.sparkcommodities.com/api/code-examples/analytics-examples.html). 
